In [1]:
import requests
from bs4 import BeautifulSoup as soup
import spacy
import json
import spacy
from spacy import displacy
from spacy.matcher import Matcher
from spacy.tokens import Span
nlp = spacy.load('en_core_web_sm')
from string import punctuation
import numbers

def get_ld_json(url: str) -> dict:
    parser = "html.parser"
    req = requests.get(url)
    page = soup(req.text, parser)
    return json.loads("".join(page.find("script", {"type":"application/ld+json"}).contents))

In [8]:
def get_name(jsonld):
    try:
        name = jsonld['name']
    except:
        name = jsonld[1]['name']
    
    lst = []
    for i, each in enumerate(name.lower().split()):
        lst.append(each.capitalize())
    name = ''.join(lst)
    
    return name

In [38]:
def get_ingredients_bonapp(jsonld):
    try:
        ingredients = jsonld['recipeIngredient']
    except:
        ingredients = jsonld[1]["recipeIngredient"]

    core = []
    for item in ingredients:
        if item.split()[0].isnumeric():
            doc = nlp(' '.join(item.split()[2:]))
        else:
            doc = nlp(item)
#         doc = nlp(item)
#         displacy.render(doc, style="dep")
        key = []
        for token in doc:
            print(token, token.dep_, token.pos_)
            if token.dep_ == "compound" or token.dep_ == "ROOT":
                key.append(token.text)
    #             print(token.text, token.pos_, token.tag_, token.dep_, token.shape_, token.is_alpha, token.is_stop, token.children, token.head)
    #             displacy.render(doc, style="dep")
        core.append(' '.join(key))
        
    for j, item in enumerate(core):
        if item == 'syrup':
            core[j] = 'simpleSyrup'
            continue
        elif len(item.split()) == 1:
            item = item.lower()
        else:
            lst = []
            for i, each in enumerate(item.lower().split()):
                if i == 0:
                    lst.append(each)
                else:
                    lst.append(each.capitalize())
            item = ''.join(lst)
        core[j] = item

    
    return core

In [85]:
def get_ingredients_allrecipes(jsonld):
    try:
        ingredients = jsonld['recipeIngredient']
    except:
        ingredients = jsonld[1]["recipeIngredient"]
        
    core = []
    measurement = ['ounce', 'ounces', 'cup', 'cups', 'tablespoon', 'tablespoons', 'fluid']
    
    for i, ingredient in enumerate(ingredients):
        lst = ingredient.split()
        while lst[0].isnumeric() or any([x == lst[0] for x in measurement]):
            lst.pop(0)
        word = ' '.join(lst)
        
        doc = nlp(word)
        for token in doc:
#             print(token, token.pos_, token.dep_)
            if token.dep_ == 'nmod':
                w = word.split()
                w.remove(token.text)
                word = ' '.join(w)
                
        w = []
        for token in word.split():
            token = ''.join(e for e in token if e.isalnum())
            w.append(token)
            
        word = ' '.join(w)
        
        
        if word.strip() not in core:
            core.append(word.strip())
            
    for j, item in enumerate(core):
        if len(item.split()) == 1:
            item = item.lower()
        else:
            lst = []
            for i, each in enumerate(item.lower().split()):
                if i == 0:
                    lst.append(each)
                else:
                    lst.append(each.capitalize())
            item = ''.join(lst)
        core[j] = item
            
    return core
    
    

In [89]:
def into_krf(jsonld):
    ingredients = get_ingredients_allrecipes(jsonld)
    name = get_name(jsonld)
    
    query = []
    for i in ingredients:
        query.append("(ingredientOf %s %s)" % (i, name))
    
    print(query)
    
    

In [90]:
jsonld = get_ld_json("https://www.allrecipes.com/recipe/221894/singapore-sling-cocktail/?internalSource=recipe%20hub&referringContentType=Search&clickId=cardslot%2021")
# core = get_ingredients(jsonld)
core = get_ingredients_allrecipes(jsonld)
print(core)
name = get_name(jsonld)
into_krf(jsonld)

['ice', 'gin', 'cherryflavoredBrandy', 'tripleSec', 'benedictineLiqueur', 'pineappleJuice', 'limeJuice', 'grenadineSyrup', 'freshPineapple', 'maraschinoCherry']
['(ingredientOf ice SingaporeSlingCocktail)', '(ingredientOf gin SingaporeSlingCocktail)', '(ingredientOf cherryflavoredBrandy SingaporeSlingCocktail)', '(ingredientOf tripleSec SingaporeSlingCocktail)', '(ingredientOf benedictineLiqueur SingaporeSlingCocktail)', '(ingredientOf pineappleJuice SingaporeSlingCocktail)', '(ingredientOf limeJuice SingaporeSlingCocktail)', '(ingredientOf grenadineSyrup SingaporeSlingCocktail)', '(ingredientOf freshPineapple SingaporeSlingCocktail)', '(ingredientOf maraschinoCherry SingaporeSlingCocktail)']
